<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/effnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

#connect drive with colab notebook/
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [9]:
!pip install -U efficientnet==0.0.4
!pip install keras-rectified-adam


Requirement already up-to-date: efficientnet==0.0.4 in /usr/local/lib/python3.6/dist-packages (0.0.4)


EfficientNetB0 - (224, 224, 3)  
EfficientNetB1 - (240, 240, 3)  
EfficientNetB2 - (260, 260, 3)  
EfficientNetB3 - (300, 300, 3)  
EfficientNetB4 - (380, 380, 3)  
EfficientNetB5 - (456, 456, 3)  
EfficientNetB6 - (528, 528, 3)  
EfficientNetB7 - (600, 600, 3)  

https://github.com/keras-team/keras/issues/8130
https://stackoverflow.com/questions/56796331/how-to-input-multiple-images-with-flow-from-dataframe-in-keras

In [10]:
#following are libraries to unpack 7z file
!pip install patool 
!pip install pyunpack
from pyunpack import Archive

In [0]:
import pandas as pd
import numpy as np
import cv2

In [12]:

#create a folder name as data
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [0]:
#path where our data is saved
path='/content/drive/My Drive/ghina/thesis/'

In [0]:
#extract the 7z file in data folder we created above
#this not only extract the data but move it from drive to colab for efficinet processing
#it will take some time
Archive(path+'ODIR-5K_Training_Images.7z').extractall('data')

In [0]:
Archive(path+'ODIR-5K_Testing_Images.7z').extractall('data')


In [0]:
#read the annottion file
annotation=pd.read_excel(path+'ODIR-5K_Training_Annotations(Updated)_V2.xlsx')

In [0]:

#normal (N), diabetes (D), glaucoma (G), cataract (C), AMD (A), hypertension (H), myopia (M) and other diseases/abnormalities (O)

In [18]:
#print first 5 rows of annotation file
#annotation=annotation.sample(frac=1)
annotation.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1


In [0]:
val=[]
for i in ['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']:
  df=annotation.sort_values(by=[i],ascending=False).head(20)
  val.append(df)
  annotation.drop(df.index,inplace=True)


In [20]:
val = pd.concat(val)
val=val.sample(frac=1)
val.head(5)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
733,735,77,Male,735_left.jpg,735_right.jpg,mild nonproliferative retinopathy,macular epiretinal membrane，mild nonproliferat...,0,1,0,0,0,0,0,1
364,365,60,Female,365_left.jpg,365_right.jpg,hypertensive retinopathy，suspected glaucoma,hypertensive retinopathy，suspected glaucoma,0,0,1,0,0,1,0,0
1434,1623,58,Female,1623_left.jpg,1623_right.jpg,pathological myopia,pathological myopia,0,0,0,0,0,0,1,0
755,757,58,Female,757_left.jpg,757_right.jpg,moderate non proliferative retinopathy,glial remnants anterior to the optic disc,0,1,0,0,0,0,0,1
3036,4174,66,Female,4174_left.jpg,4174_right.jpg,mild nonproliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0


In [21]:
test=pd.read_csv(path+'XYZ_ODIR.csv')
print(len(test))
test.head()


500


,ID,N,D,G,C,A,H,M,O
0,937,0,0,0,0,0,0,0,0
1,967,0,0,0,0,0,0,0,0
2,988,0,0,0,0,0,0,0,0
3,995,0,0,0,0,0,0,0,0
4,1000,0,0,0,0,0,0,0,0


In [22]:
print(len(annotation))
print(len(val))
print(len(test))

3340
160
500


In [23]:
left_test=[]
right_test=[]
ID=[]
for i in test['ID']:
  ID.append(i)
  left_test.append(str(i)+'_left.jpg')
  right_test.append(str(i)+'_right.jpg')

name_df=pd.DataFrame(zip(ID,left_test,right_test),columns=['ID','Left-Fundus','Right-Fundus'])
name_df.head()
result = pd.merge(name_df, test, on='ID')
result.head()

,ID,Left-Fundus,Right-Fundus,N,D,G,C,A,H,M,O
0,937,937_left.jpg,937_right.jpg,0,0,0,0,0,0,0,0
1,967,967_left.jpg,967_right.jpg,0,0,0,0,0,0,0,0
2,988,988_left.jpg,988_right.jpg,0,0,0,0,0,0,0,0
3,995,995_left.jpg,995_right.jpg,0,0,0,0,0,0,0,0
4,1000,1000_left.jpg,1000_right.jpg,0,0,0,0,0,0,0,0


In [0]:
for i in ['G',	'C',	'A',	'H',	'M']:
  duplicate=annotation[annotation[i]==1]
  annotation=annotation.append([duplicate]*2,ignore_index=True)

In [25]:
len(annotation)

4976

In [0]:
img_shape=380

In [0]:
#https://www.kaggle.com/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [0]:
def load_ben_color(image, sigmaX=10):
    #image = cv2.imread(path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (img_shape, img_shape))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    
    
    return image

In [0]:
from keras_preprocessing.image import ImageDataGenerator

train_data_gen= ImageDataGenerator(
    horizontal_flip=True,
    #vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    #brightness_range=[0.5, 1.5],
    preprocessing_function=load_ben_color
)

val_data_gen= ImageDataGenerator(
    horizontal_flip=True,
    #vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    #brightness_range=[0.5, 1.5],
    preprocessing_function=load_ben_color
)



test_data_gen= ImageDataGenerator(
    rescale=1/255
    
)

In [0]:
def get_flow_from_dataframe(generator, dataframe,directory_path,
                            image_shape=(img_shape, img_shape),
                            #subset='training',
                             batch_size=35):
    train_generator_1 = generator.flow_from_dataframe(dataframe,
                                                      directory=directory_path,
                                                      target_size=(img_shape,img_shape),
                                                      x_col="Left-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )

    train_generator_2 = generator.flow_from_dataframe(dataframe, 
                                                      directory=directory_path,
                                                      target_size=(img_shape,img_shape),
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )
    while True:
        x_1 = train_generator_1.next()
        x_2 = train_generator_2.next()

        yield [x_1[0], x_2[0]], x_1[1]

In [0]:
batch_size=5
train_gen = get_flow_from_dataframe(train_data_gen, annotation, image_shape=(img_shape, img_shape),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
valid_gen = get_flow_from_dataframe(val_data_gen, val, image_shape=(img_shape, img_shape),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
test_gen = get_flow_from_dataframe(test_data_gen, result, image_shape=(img_shape, img_shape),
                                        directory_path="data/ODIR-5K_Testing_Images/",batch_size=batch_size)

In [50]:
img_shape

380

In [0]:
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate,Input
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:

from efficientnet import EfficientNetB4
import keras
left_input=Input(shape=(img_shape,img_shape,3))
right_input=Input(shape=(img_shape,img_shape,3))

left_model = EfficientNetB5(input_shape =  (img_shape,img_shape,3), include_top = False, weights = 'imagenet',input_tensor=left_input)
right_model = EfficientNetB5(input_shape =  (img_shape,img_shape,3), include_top = False, weights = 'imagenet',input_tensor=right_input)

left_model.trainable = True
right_model.trainable = True

In [0]:
con = concatenate([left_model.output, right_model.output])
gap_dr = Dropout(0.5)(con)
GAP= GlobalAveragePooling2D()(gap_dr)
fc1=Dense(128, activation = 'relu')(GAP)
fc2=Dense(64, activation = 'relu')(fc1)
fc3=Dense(32, activation = 'relu')(fc2)
out = Dense(8, activation = 'sigmoid')(fc3)

In [0]:

for layer in right_model.layers:
        layer.name = layer.name + '_right'
for layer in left_model.layers:
  layer.name = layer.name + '_left'

left_model.name="left_eff"
right_model.name="right_eff"

In [0]:
model =Model(inputs=[left_input, right_input], outputs=out)


In [0]:
# from keras.utils import plot_model
# plot_model(model)

In [0]:
# import keras.backend as K
# def focal_loss(y_true, y_pred, alpha = 0.75, gamma = 3):
#   alpha_factor = K.ones_like(y_true) * alpha
#   alpha_factor = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
#   focal_weight = tf.where(K.equal(y_true, 1), 1 - y_pred, y_pred)
#   focal_weight = alpha_factor * focal_weight ** gamma
#   loss = focal_weight * K.binary_crossentropy(y_true, y_pred)
#   return loss

In [0]:
from keras_radam import RAdam

model.compile(RAdam(),loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
step_size_train=len(annotation)/batch_size
step_size_val=len(val)/batch_size


In [0]:
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.000001, verbose=1)
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')


In [61]:
model.fit_generator(train_gen, epochs=10,
                        steps_per_epoch=step_size_train,
                        use_multiprocessing=True,
                        validation_data=valid_gen,
                        validation_steps=step_size_val,
                        verbose=2,callbacks=[reduce_lr,checkpoint]
                   # class_weight=[0.5,0.5,1.5,1.5,1.5,2.5,1.5,0.5]
                    
                        )

Epoch 1/10
Found 160 validated image filenames.
Found 160 validated image filenames.


Found 4976 validated image filenames.
Found 4976 validated image filenames.
 - 1665s - loss: 0.2799 - acc: 0.8984 - val_loss: 3.4110 - val_acc: 0.7070

Epoch 00001: val_loss improved from inf to 3.41099, saving model to model.h5
Epoch 2/10
 - 1499s - loss: 0.2471 - acc: 0.9133 - val_loss: 1.4515 - val_acc: 0.7055

Epoch 00002: val_loss improved from 3.41099 to 1.45148, saving model to model.h5
Epoch 3/10
 - 1523s - loss: 0.2648 - acc: 0.9052 - val_loss: 0.8888 - val_acc: 0.6953

Epoch 00003: val_loss improved from 1.45148 to 0.88880, saving model to model.h5
Epoch 4/10
 - 1517s - loss: 0.2737 - acc: 0.9022 - val_loss: 0.5919 - val_acc: 0.7367

Epoch 00004: val_loss improved from 0.88880 to 0.59188, saving model to model.h5
Epoch 5/10
 - 1493s - loss: 0.2748 - acc: 0.9003 - val_loss: 0.8534 - val_acc: 0.7367

Epoch 00005: val_loss did not improve from 0.59188
Epoch 6/10
 - 1483s - loss: 0.2750 - acc: 

In [62]:
y_pred = model.predict_generator(valid_gen,steps=step_size_val)
y_pred1=y_pred.copy()
y_pred1=y_pred1.flatten()
y_pred1[y_pred1>=0.5] = 1
y_pred1[y_pred1<0.5] = 0
y_pred1=y_pred1.astype(int)

Found 160 validated image filenames.
Found 160 validated image filenames.


In [63]:
y_true=val.iloc[:,-8::].values.flatten()
y_true.shape

(1280,)

In [70]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,cohen_kappa_score,roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

print(cohen_kappa_score(y_true, y_pred1))
print(f1_score(y_true, y_pred1,average='micro'))
print(roc_auc_score(y_true, y_pred1))

0.0
0.82890625
0.5


In [0]:
# target_names = ['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']
# print(classification_report(y_true, y_pred1, target_names=target_names))

In [66]:
y_test= model.predict_generator(test_gen,steps=len(test)/batch_size)


Found 500 validated image filenames.
Found 500 validated image filenames.


In [0]:
for i,j in enumerate(['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']):
  test[j]=y_test[:,i]

In [68]:
test.head()

,ID,N,D,G,C,A,H,M,O
0,937,0.137506,0.234278,0.219162,0.191149,0.204718,0.137879,0.20152,0.263012
1,967,0.137506,0.234278,0.219162,0.191149,0.204718,0.137879,0.20152,0.263012
2,988,0.137506,0.234278,0.219162,0.191149,0.204718,0.137879,0.20152,0.263012
3,995,0.137506,0.234278,0.219162,0.191149,0.204718,0.137879,0.20152,0.263012
4,1000,0.137506,0.234278,0.219162,0.191149,0.204718,0.137879,0.20152,0.263012


In [0]:
test.to_csv('/content/TAC_ODIR.csv',index=False)

In [0]:
keras.backend.clear_session()
